### IndPenSim Data websites 
Addition details can be found at:<a href='http://www.industrialpenicillinsimulation.com/'> www.industrialpenicillinsimulation.com</a>
Data is downloaded from : <a href='https://data.mendeley.com/datasets/pdnjz7zz5x/1'> Mendeley data Website</a>

In [ ]:
## Import necessary packages
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd

## Data Summary

In [ ]:
data = pd.read_csv('data/100_Batches_IndPenSim_V3.csv'); 

variable_raman_data = np.split(data, [data.columns.get_loc("2400")], axis=1); 
variable_data = variable_raman_data[0]; 
print(variable_data.columns); 
variable_data


## Data Processing
### Split data into spectral and process data
Splitting the data into process measurement data and Raman spectroscopy data

In [ ]:
variable_data.describe(); 
### Reable Batch ID's correctly
variable_data = variable_data.rename(columns={'2-PAT control(PAT_ref:PAT ref)': 'Batch reference(Batch_ref:Batch ref)','Batch reference(Batch_ref:Batch ref)':'2-PAT control(PAT_ref:PAT ref)'}); 

cleanups = ["Ammonia", "PAT", "Batch ID", "Raman", "RPM", "Fault flag", "Weight", "evolution"]; 
for name in cleanups:
    variable_data = variable_data[variable_data.columns.drop(list(variable_data.filter(regex=name)))]; 

variable_data = variable_data.dropna(thresh=variable_data.shape[0]*0.6,axis=1); # Remove rows more than 60% empty
variable_data.drop(columns=variable_data.columns[variable_data.eq(0.0).mean()>0.85]); # Remove cols with more than 85% 0s

variable_data = variable_data.fillna(0) # Fill NA
variable_data.to_csv("data/clean_data.csv", sep=',', index=False); 

In [ ]:
cols = variable_data.columns.to_list(); 

with open('data/V3_heads.jsonc', 'w') as f:
    f.write("{\n"); 
    for idx,line in enumerate(cols):
        f.write("\t\"%s::none\":\"%s\",\n" % (idx,line)); 
    f.write("}"); 


print(cols); 
variable_data

In [ ]:
corr_matrix = variable_data.corr().abs(); 
corr_matrix

In [ ]:
# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool)); 

to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]; 
print(to_drop)